# Main figure generation and analysis code

This code is used to generate the figures and run the analysis for all MSOT figures in the IPASC Multicentre Paper. Before running this script, please make sure to download the data from the repository specified in the README, and update the file paths in the second code block to ensure that this code runs correctly. 

In [ ]:
import numpy as np
import patato as pat
import matplotlib.pyplot as plt

from utils import remove_tick_labels, add_scalebar, setup_matplotlib
from tqdm.auto import tqdm
import pandas as pd
import matplotlib.transforms as transforms

from datetime import datetime
from pathlib import Path

setup_matplotlib(300)

In [ ]:
#### UPDATE THE DATA PATHS HERE WHEN RUNNING THE CODE. ###

data_directory = Path("/media/telse/Extreme SSD/Papers/IPASCMultiCentre/Fixed/Data3")
example_file = Path("/media/telse/Extreme SSD/Papers/IPASCMultiCentre/Fixed/Data3/Scan_4.hdf5")

if not data_directory.exists() or not example_file.exists():
    raise ValueError("Data directory and example files do not exist. Please download data from the specified repository and provide the correct path (see README).")

In [ ]:
masks = np.load("intermediate_results/translated_masks.npz")

In [ ]:
def sort_key(s):
    return int(s.stem.split("_")[-1])

In [ ]:
data_files = sorted(data_directory.glob("**/Scan_*.hdf5"), key=sort_key)
spectra = []

for file in tqdm(data_files):
    pa = pat.PAData.from_hdf5(file)
    # print(pa.get_scan_reconstructions().keys())
    if "Clear" in pa.get_scan_name() or ('Reference Backprojection', '0') not in pa.get_scan_reconstructions(): # pyright: ignore[reportOperatorIssue]
        print(file, pa.get_scan_name())
        continue
    pa.set_default_recon(('Reference Backprojection', '0')) # pyright: ignore[reportArgumentType]
    rec1 = pa.get_scan_reconstructions()
    
    rec_data = np.squeeze(rec1.raw_data) # type: ignore
    # rec_data /= pa.get_overall_correction_factor()[:, :, None, None]
    mask = masks[str(file)]
    for i in range(mask.shape[0]):
        m = mask[i] & np.all(rec_data[i] > 0, axis=0)
        spectrum = rec_data[i].T[m.T].T
        spectra.append({"PA": np.median(spectrum, axis=1), "Wavelength": pa.get_wavelengths(), "Scan": pa.get_scan_name(), "File": file})


In [ ]:
df = pd.DataFrame(spectra)

In [ ]:
df["Operator"] = df["Scan"].apply(lambda x:" ".join(x.split("_")[:-1]))
df["Operator"] = df["Operator"].apply(lambda x:x.replace(" Nigrosin", ""))
df["Replicate"] = df["Scan"].apply(lambda x:x.split("_")[-1])

# display(df[df["Operator"] == "LW05"].iloc[0]["File"])

del df["Scan"]
del df["File"]

In [ ]:
df_g = df.groupby(["Operator", "Replicate"]).mean().reset_index()
df_g.tail()

In [ ]:
date = datetime.now().strftime("%Y%m%d")

'20250919'

In [ ]:
df_g["Operator"] = df_g["Operator"].apply(lambda x: x.split()[0])
df_long = df_g.explode(["PA", "Wavelength"])
df_long.to_excel(f"intermediate_results/{date}_ipasc_multicentre_ipasc.xlsx")

## Separate out the LPM/HPM

In [ ]:
df_details = pd.read_excel("scandetails.ods")[["Reference", "Recipe"]]

In [ ]:
df_recipe = df_g.merge(df_details, left_on="Operator", right_on="Reference")

In [ ]:
df_summarised = df_recipe.groupby("Operator").agg({"Recipe": "first", 
                                                   "PA": lambda x: {"Mean": np.mean(np.stack(x), axis=0), 
                                                                    "Std": np.std(np.stack(x), axis=0)}, 
                                                   "Wavelength": "first"})
df_summarised = df_summarised.drop(index="LW21")
df_summarised = df_summarised.drop(index="LW04")
df_summarised = pd.concat([df_summarised, df_summarised["PA"].apply(pd.Series)], axis=1)
del df_summarised["PA"]
df_summarised["PA"] = df_summarised["Mean"]
df_summarised["PA Std"] = df_summarised["Std"]
del df_summarised["Mean"]
del df_summarised["Std"]
df_summarised["CoV"] = df_summarised["PA Std"] / df_summarised["PA"]
df_summarised.head()

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2))

for recipe, rec_grp in df_summarised.groupby("Recipe"):
    wls = rec_grp["Wavelength"].iloc[0]
    spectra = np.stack(rec_grp["PA"]) # type: ignore
    mean_spec = np.mean(spectra, axis=0)
    se_spec = np.std(spectra, axis=0) / np.sqrt(spectra.shape[0])
    
    ax.plot(wls, mean_spec, label=recipe)
    ax.fill_between(wls, mean_spec-se_spec, mean_spec + se_spec, alpha=0.3)
ax.set_xlabel("Wavelength (nm)")

## Plot the individual spectra

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2))

wls = df_summarised["Wavelength"].iloc[0]
spectra = np.stack(df_summarised["PA"]) # type: ignore
mean_spec = np.mean(spectra, axis=0)
se_spec = np.std(spectra, axis=0) / np.sqrt(spectra.shape[0])

plt.plot(wls, mean_spec, label="Mean", c="C2")
plt.fill_between(wls, mean_spec - se_spec, mean_spec + se_spec, alpha=0.1, label="SE", facecolor="C2")

for operator, rec_grp in df_summarised.iterrows():
    wls = rec_grp["Wavelength"]
    ax.plot(wls, rec_grp["PA"], alpha=0.1, c="k")
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("PA signal (a.u.)")
ax.legend()
# ax.set_ylim([0, None])
fig.savefig("figures/AllSpectra_Batch3.png", dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2))

for recipe, rec_grp in df_summarised.groupby("Recipe"):
    wls = rec_grp["Wavelength"].iloc[0]
    spectra = np.stack(rec_grp["CoV"]) # type: ignore
    mean_spec = np.mean(spectra, axis=0)
    se_spec = np.std(spectra, axis=0) / np.sqrt(spectra.shape[0])
    
    ax.plot(wls, mean_spec, label=recipe)
    ax.fill_between(wls, mean_spec-se_spec, mean_spec + se_spec, alpha=0.3)
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Coefficient of Variation")
ax.legend()
# ax.set_ylim([0, None])
fig.savefig("figures/covLPMvsHPM_Batch3.png", dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2))

wls = df_summarised["Wavelength"].iloc[0]
spectra = np.stack(df_summarised["CoV"]) # type: ignore
mean_spec = np.mean(spectra, axis=0)
se_spec = np.std(spectra, axis=0) / np.sqrt(spectra.shape[0])

plt.plot(wls, mean_spec, label="Mean", c="C2")
plt.fill_between(wls, mean_spec - se_spec, mean_spec + se_spec, alpha=0.1, label="SE", facecolor="C2")

for operator, rec_grp in df_summarised.iterrows():
    wls = rec_grp["Wavelength"]
    ax.plot(wls, rec_grp["CoV"], alpha=0.1, c="k")
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Coefficient of Variation")
ax.legend()
# ax.set_ylim([0, None])
fig.savefig("figures/AllCoVSpectra_Batch3.png", dpi=300)
plt.show()

In [ ]:
df_summarised["CoV"].apply(lambda x: x[0]).argsort()

In [ ]:
df_summarised.iloc[1]

In [ ]:
pa = pat.PAData.from_hdf5(example_file) # type: ignore

In [ ]:
fig = plt.figure(figsize=(6.27, 1.5*10.5/10))

subfigs = fig.subfigures(2, 3, width_ratios=[1, 1, 1.5], height_ratios=[0.5,10])
ax3, ax2, ax1 = [s.subplots() for s in subfigs[1]]

ax3.axis("off")

for recipe, rec_grp in df_summarised.groupby("Recipe"):
    print(recipe, rec_grp.shape[0], 100*rec_grp["CoV"].apply("mean").mean(), 100*rec_grp["CoV"].apply("mean").std()/np.sqrt(6)) # type: ignore
    wls = rec_grp["Wavelength"].iloc[0]
    spectra = np.stack(rec_grp["PA"]) # type: ignore
    mean_spec = np.mean(spectra, axis=0)
    se_spec = np.std(spectra, axis=0) / np.sqrt(spectra.shape[0])
    
    ax1.plot(wls, mean_spec, label=recipe)
    ax1.fill_between(wls, mean_spec-se_spec, mean_spec + se_spec, alpha=0.3)
ax1.set_xlabel("Wavelength (nm)")
ax1.legend()

pa.set_default_recon()
rec = pa.get_scan_reconstructions()
print(pa.get_wavelengths()[0])
im = rec.imshow(ax=ax2, scalebar=False) # type: ignore
ax2.axis("on")
add_scalebar(0.01, ax=ax2)
remove_tick_labels(np.array([ax2]))

trans = transforms.blended_transform_factory(subfigs[0, 0].transSubfigure, fig.transFigure)
fig.text(0, 1, "A", transform=trans, fontweight="bold", ha="left", va="top", fontsize="large")

trans = transforms.blended_transform_factory(subfigs[0, 1].transSubfigure, fig.transFigure)
fig.text(0, 1, "B", transform=trans, fontweight="bold", ha="left", va="top", fontsize="large")

trans = transforms.blended_transform_factory(subfigs[0, 2].transSubfigure, fig.transFigure)
fig.text(0, 1, "C", transform=trans, fontweight="bold", ha="left", va="top", fontsize="large")

ax1.set_ylabel("PA intensity (a.u.)")
plt.colorbar(im, ax=ax2) # type: ignore
plt.savefig("figures/example_image.png", dpi=300)
plt.savefig("figures/example_image.svg", dpi=300)
plt.show()